In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import configparser

In [3]:
config = configparser.ConfigParser()
config.read('./Data/estados_de_cuenta/config.ini')

# Reading the data inside the xml
# file to a variable under the name 
# data
file_name='05_2024'
data_address=config['DATA']['ADDRESS']

with open(f'{data_address}/xml/{file_name}.xml', 'r') as f:
    data = f.read()

f.close()
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object 
Bs_data = BeautifulSoup(data, "xml")
 
# Finding all instances of tag 
# `unique`
b_unique = Bs_data.find_all('DG:MovimientosDelCliente')
 



In [4]:
dict_form = lambda record: { "fecha" : record.get("fecha"), "descripcion" : record.get("descripcion") , "importe" : record.get("importe"), "ingreso": -1}
records_dict_list = [ dict_form(record) for record in b_unique ]

In [5]:
df = pd.DataFrame(records_dict_list)
csv_file = f"{data_address}/csv/{file_name}.csv"
if os.path.exists(csv_file):
    print("File already exist.")
else:
    df.to_csv(csv_file)
month_data = pd.read_csv(csv_file)


File already exist.


In [6]:
total_money = lambda x: month_data[ month_data["ingreso"] == x]["importe"].sum() 
ingresos = total_money(1)
egresos = total_money(-1)
print("Ingresos: ",ingresos)
print("Egresos: ",egresos)

Ingresos:  58177.47
Egresos:  55443.52


In [7]:
month_data[ month_data["ingreso"] == 1].sort_values("importe",ascending=False)

,Unnamed: 0,fecha,descripcion,importe,ingreso
2,2,2024-05-06T12:00:00,INSTRUYE IRVTUS3NXXX CAMPO 72 SI 0611929,38549.50,1
14,14,2024-05-10T12:00:00,PARA NU 0271195,9568.06,1
24,24,2024-05-31T12:00:00,2700 EGRESOS SPEI SVD 5594424,7383.66,1
0,0,2024-05-03T12:00:00,FINSUS 0240503,2402.35,1
17,17,2024-05-13T12:00:00,74208474131000017927314SM NORD VPNCOMGB,273.90,1
